In [ ]:
import numpy
from itertools import combinations
import csv
import pandas
import math
from sys import getsizeof
import timeit

In [ ]:
df = pandas.read_csv('DT_book.csv', header=0)
#df['A']

# Optimization

## Information Gain

In [ ]:
def info_gain(df, pred):
    #names=df.columns.get_values().tolist()
    unique = df[pred].unique()
    counts = df[pred].value_counts()
    info=0
    for u in unique:
        total_count= df[pred].shape[0]
        count_us= df[df[pred] == u].shape[0]
        info = info+(-count_us/total_count)*math.log2(count_us/total_count)
    #print("Info",info)
    
    gain = []
    for d in df.columns:
        if d!=pred:
            uniquew = df[d].unique()
            infoA=0
            for w in uniquew:
                info1 = 0
                dw = df[df[d] == w]
                #print(dw, "\n---")
                for u in unique:
                    count_uw = dw[dw[pred]==u].shape[0]
                    count_total = dw.shape[0]
                    if count_uw!=0:
                        info1= (-count_uw/count_total*math.log2(count_uw/count_total))
                        infoA=infoA+(info1*(dw.shape[0]/df[d].shape[0]))
                    #print(d,w,u, count_uw,count_total, info1)
                #infoA=infoA+info1
            #print(d, infoA)
            gain.append(info-infoA)
        else:
            gain.append(-1)
    #print(gain)
    #i=gain.index(max(gain))
    return gain

In [ ]:
## testing
import math
print(df)
print("-----------------------")
info_gain(df, 'venomous')

### Gain Ratio

In [ ]:
def gain_ratio(df, pred):
    gains=info_gain(df,pred)
    split_info=[]
    for d in df.columns:
        if d!=pred:
            uniquew = df[d].unique()
            info1 = 0
            for w in uniquew:
                dw = df[df[d] == w]
                count_uw=dw.shape[0]
                total_count= df[d].shape[0]
                info1= info1+((-count_uw/total_count)*math.log2(count_uw/total_count))
            #print(d,count_uw,total_count, info1)
            split_info.append(info1)
        else:
            split_info.append(0)
    gain_ratio=numpy.asarray(gains)/numpy.asarray(split_info)
    return gain_ratio.tolist()

In [ ]:
gain_ratio(df,'venomous')

### Gini Optimization

In [ ]:
def gini(df, pred):
    unique = df[pred].unique()
    counts = df[pred].value_counts()
    gini=0
    for u in unique:
        total_count= df[pred].shape[0]
        count_us= df[df[pred] == u].shape[0]
        gini = gini+math.pow((count_us/total_count),2)
    gini=1-gini
    ginis=[]
    
    for d in df.columns:
        if d!=pred:
            uniqued = df[d].unique()
            #print(d, uniqued)
            for i in range(1, len(uniqued)):
                comb = combinations(uniqued, i)
                combo_list = list(comb)
                for combo in combo_list:
                    gini_y=0.0
                    gini_n=0.0
                    #print("Combo", combo)             
                    dwy = df[df[d].isin(combo)]
                    dwn = df[df[d].isin(combo)==False]
                    #print(dwy)
                    for u in unique:
                        gini_y=gini_y+math.pow((dwy[dwy[pred]==u].shape[0]/dwy.shape[0]),2)
                        gini_n=gini_n+math.pow((dwn[dwn[pred]==u].shape[0]/dwn.shape[0]),2)
                        #print(combo, u, dwy.shape[0],dwy[dwy[pred]==u].shape[0], gini_y)
                    gini_y= (1-gini_y)*dwy.shape[0]/df[d].shape[0]

                    gini_n=(1-gini_n)*dwn.shape[0]/df[d].shape[0]
                    #print(dwy.shape[0],dwn.shape[0],df[d].shape[0], gini_y+gini_n)
                    ginis.append((d, combo, gini_y+gini_n))
    return ginis

In [ ]:
gini(df,'buys computer')

# Splitting

In [ ]:
def get_split_infogain(df, pred):
    header = list(df)
    i = info_gain(df,pred)
    position = i.index(max(i))
    attr = header[position]
    values = df[attr].unique().tolist()
    sub_dataset=[]
    for val in values:
        sub_df = df[df[attr]==val]
        sub_dataset.append((val, sub_df))
    return attr, sub_dataset

##TESTING
get_split_infogain(df,'buys computer')

In [ ]:
def get_split_gainratio(df, pred):
    header = list(df)
    i = gain_ratio(df,pred)
    position = i.index(max(i))
    attr = header[position]
    values = df[attr].unique().tolist()
    sub_dataset=[]
    for val in values:
        sub_df = df[df[attr]==val]
        sub_dataset.append((val, sub_df))
    return attr, sub_dataset

##TESTING
get_split_gainratio(df,'Dx:Cancer')

In [ ]:
def get_split_gini(df, pred):
    g = gini(df,pred)
    min_value= g[0][2]
    min_split = g[0][1]
    min_attr = g[0][0]
    for element in g:
        if element[2]<min_value:
            min_value=element[2]
            min_split = element[1]
            min_attr=element[0]
    mask = df[min_attr].isin(min_split)
    label=min_attr
    left = df[mask]
    right = df[~mask]
    return label,left,right
##Testing
get_split_gini(df_train,'Dx:Cancer')

# Decision Tree

In [ ]:
class TreeNode(object):
    def __init__(self, value, children = []):
        self.value = value
        self.children = children

    def __str__(self, level=0):
        ret = "\t"*level+repr(self.value)+"\n"

        for child in self.children:
            ret += child.__str__(level+1)
        return ret

    def __repr__(self):
        return '<tree node representation>'

In [ ]:
def decision_tree_gini(df,pred):
    temp=df
    root = TreeNode(["root"],[])
    root.children=[]
    curr=root  
    index = 0
    to_build = []
    
    label,left,right = get_split_gini(temp,pred)
    #print(label)
    l_label = left[label].unique().tolist()
    r_label = right[label].unique().tolist()
    #print(l_label, r_label)
    root.value= [label]
    left_node = TreeNode([l_label],[])
    right_node= TreeNode([r_label],[])
    
    del left[label]
    del right[label]

    to_build.append([left_node, left])
    to_build.append([right_node, right])
    
    curr.children.append(left_node)
    curr.children.append(right_node)
    
    while(len(to_build)>0):
        
        element=to_build.pop(0)
        curr = element[0]
        temp = element[1]
        if (temp.shape[1]<=2): 
            pred_list = list(temp[pred])
            del temp[pred]
            label = list(temp)
            leaf_list=temp.values.tolist()
            curr.value.append(label[0])
            
            for i in range(len(leaf_list)):
                node = TreeNode([leaf_list[i]],[])
                curr.children.append(node)
                node.value.append(pred)
                pred_node = TreeNode([pred_list[i]],[])
                node.children.append(pred_node)
                
            
        else:
            label,left,right = get_split_gini(temp,pred)
            curr.value.append(label)

            l_label = left[label].unique().tolist()
            r_label = right[label].unique().tolist()
            print(label, l_label, r_label)
            print(left, right)
            left_node = TreeNode([l_label],[])
            right_node= TreeNode([r_label],[])

            del left[label]
            del right[label]

            to_build.append([left_node, left])
            to_build.append([right_node, right])

            curr.children.append(left_node)
            curr.children.append(right_node)
        #print(to_build)
    return(root)

In [ ]:
ginitree= decision_tree_gini(df_train,'buys computer')
print(ginitree)

In [ ]:
def decision_tree_infogain(df,pred):
    temp=df
    root = TreeNode(["root"],[])
    root.children=[]
    curr=root  
    index = 0
    to_build = []
    
    label, part_data = get_split_infogain(df,pred)
    root.value= [label]
    for dataset in part_data:
        dataset[1][label].unique()
        node = TreeNode([dataset[0]],[])
        sub_data = dataset[1]
        del sub_data[label]
        to_build.append([node, sub_data])
        curr.children.append(node)
    
    while(len(to_build)>0):
        element=to_build.pop(0)
        curr = element[0]
        temp = element[1]
        if (temp.shape[1]<=2): 
            pred_list = list(temp[pred])
            del temp[pred]
            label = list(temp)
            leaf_list=temp.values.tolist()
            curr.value.append(label[0])
            
            for i in range(len(leaf_list)):
                node = TreeNode([leaf_list[i]],[])
                curr.children.append(node)
                node.value.append(pred)
                pred_node = TreeNode([pred_list[i]],[])
                node.children.append(pred_node)
                
            
        else:
            label, part_data = get_split_infogain(temp,pred)
            curr.value.append(label)
            for dataset in part_data:
                dataset[1][label].unique()
                node = TreeNode([dataset[0]],[])
                sub_data = dataset[1]
                del sub_data[label]
                to_build.append([node, sub_data])
                curr.children.append(node)
    return(root)

In [ ]:
infotree = decision_tree_infogain(df,"buys computer")

In [ ]:
def decision_tree_gainratio(df,pred):
    temp=df
    root = TreeNode(["root"],[])
    root.children=[]
    curr=root  
    index = 0
    to_build = []
    
    label, part_data = get_split_gainratio(df,pred)
    root.value= [label]
    for dataset in part_data:
        a = dataset[1][label]
        a.unique()
        node = TreeNode([dataset[0]],[])
        sub_data = dataset[1]
        del sub_data[label]
        to_build.append([node, sub_data])
        curr.children.append(node)
    
    while(len(to_build)>0):
        element=to_build.pop(0)
        curr = element[0]
        temp = element[1]
        if (temp.shape[1]<=2): 
            pred_list = list(temp[pred])
            del temp[pred]
            label = list(temp)
            leaf_list=temp.values.tolist()
            curr.value.append(label[0])
            
            for i in range(len(leaf_list)):
                node = TreeNode([leaf_list[i]],[])
                curr.children.append(node)
                node.value.append(pred)
                pred_node = TreeNode([pred_list[i]],[])
                node.children.append(pred_node)
                
            
        else:
            label, part_data = get_split_gainratio(temp,pred)
            curr.value.append(label)
            for dataset in part_data:
                dataset[1][label].unique()
                node = TreeNode([[dataset[0]]],[])
                sub_data = dataset[1]
                del sub_data[label]
                to_build.append([node, sub_data])
                curr.children.append(node)
    return(root)

In [ ]:
rtree = decision_tree_gainratio(df,'venomous')
print(rtree)

# Prediction

In [ ]:
def predict(tree, sample):
    curr =tree
    #print(curr.children[0].value[0])
    while(type(curr.children[0].value[0])!=list):
        currattr = curr.value[-1]
        sample_attr_val = sample[currattr]
        #print(currattr)
        for i in range(len(curr.children)):
            attr_value=curr.children[i].value
            #print(attr_value)
            if sample_attr_val in attr_value:
                curr = curr.children[i]
                #print("same")
                break
                
    
    
    if (len(curr.children)>1):
        #print("more")
        currattr = curr.value[-1]
        sample_attr_val = sample[currattr]
        for i in range(len(curr.children)):
            attr_value=curr.children[i].value
            #print(attr_value)
            if sample_attr_val in attr_value[0]:
                curr = curr.children[i]
                #print("same")
                break
    else:
        curr= curr.children[0]
        
    pred=curr.children[0]   
    value=pred.value[0]
    return value
    

In [ ]:
accuracy = 0
for s in samples:
    p = predict(infotree,s)
    if p == (s['buys computer']):
        accuracy+=1
print (accuracy/df.shape[0])

## Testing Trainining Split

In [ ]:
df = pandas.read_csv('DT_book.csv', header=0)
header=list(df)
distinct_value ={}
for h in header:
    u=df[h].unique()
    distinct_value[h]=u.tolist()
df_train = pandas.DataFrame(columns=header)
df_test = pandas.DataFrame(columns=header)
samples = []
for index, row in df.iterrows():
    if (len(distinct_value)==0):
        df_test.loc[len(df_test)] = row
        #if (df_train.shape[0]<0.7*df.shape[0]):
        #    df_train.loc[len(df_train)] = row
        #else:
        #    df_test.loc[len(df_test)] = row
    else:
        df_train.loc[len(df_train)] = row
    for h in header:
        val=row[h]
        if h in distinct_value:
            values = distinct_value[h]
            if val in values:
                values.remove(val)
                distinct_value[h]=values
            if len(values)==0:
                del distinct_value[h]
df_train.to_csv("train.csv", sep=',')
df_test.to_csv("test.csv", sep=',')

In [ ]:
start_time = timeit.default_timer()
itree = decision_tree_infogain(df_train, 'buys computer')
elapsed = timeit.default_timer() - start_time
size = getsizeof(itree)
print(elapsed, size)

accuracy = 0
for i, s in df_test.iterrows():
    p = predict(infotree,s)
    if p == (s['buys computer']):
        accuracy+=1
print (accuracy/df.shape[0])

In [ ]:
start_time = timeit.default_timer()
grtree = decision_tree_gainratio(df_train,'buys computer')
elapsed = timeit.default_timer() - start_time
size = getsizeof(grtree)
print(elapsed, size)
print(grtree)

In [ ]:
start_time = timeit.default_timer()
grtree = decision_tree_gini(df[:12],'buys computer')
elapsed = timeit.default_timer() - start_time
size = getsizeof(itree)
print(elapsed, size)

print(grtree)

In [ ]:
df = pandas.read_csv('DT_test.csv', header=0)
header=list(df)
distinct_value ={}
for h in header:
    u=df[h].unique()
    distinct_value[h]=u.tolist()
df_train = pandas.DataFrame(columns=header)
df_test = pandas.DataFrame(columns=header)
samples = []
for index, row in df.iterrows():
    if (len(distinct_value)==0):
        df_test.loc[len(df_test)] = row
        #if (df_train.shape[0]<0.7*df.shape[0]):
        #    df_train.loc[len(df_train)] = row
        #else:
        #    df_test.loc[len(df_test)] = row
    else:
        df_train.loc[len(df_train)] = row
    for h in header:
        val=row[h]
        if h in distinct_value:
            values = distinct_value[h]
            if val in values:
                values.remove(val)
                distinct_value[h]=values
            if len(values)==0:
                del distinct_value[h]
df_train.to_csv("train.csv", sep=',')
df_test.to_csv("test.csv", sep=',')


In [ ]:
start_time = timeit.default_timer()
itree = decision_tree_infogain(df_train, 'In College')
print(itree)
elapsed = timeit.default_timer() - start_time
size = getsizeof(itree)
print(elapsed, size)

accuracy = 0
for i, s in df_test.iterrows():
    p = predict(itree,s)
    print(p)
    if p == (s['In College']):
        accuracy+=1
print (accuracy/df.shape[0])

In [ ]:
print(df)
print(df_train)
start_time = timeit.default_timer()
grtree = decision_tree_gainratio(df, 'In College')
elapsed = timeit.default_timer() - start_time
size = getsizeof(grtree)
print(elapsed, size)
print(grtree)

In [ ]:
start_time = timeit.default_timer()
grtree = decision_tree_gini(df, 'In College')
elapsed = timeit.default_timer() - start_time
size = getsizeof(grtree)
print(elapsed, size)
print(grtree)

# Large DataSets

In [531]:
df = pandas.read_csv('breastCancer.csv', header=0)
#df = df[['Age', 'Num of pregnancies','Smokes','Hormonal Contraceptives', 'STDs', 'Dx:Cancer']]
del df['id']
header = list(df)
distinct_value ={}
for h in header:
    u=df[h].unique()
    distinct_value[h]=u.tolist()
df_train = pandas.DataFrame(columns=header)
df_test = pandas.DataFrame(columns=header)
samples = []
for index, row in df.iterrows():
    if (len(distinct_value)==0):
        df_test.loc[len(df_test)] = row
        #if (df_train.shape[0]<0.7*df.shape[0]):
        #    df_train.loc[len(df_train)] = row
        #else:
        #    df_test.loc[len(df_test)] = row
    else:
        df_train.loc[len(df_train)] = row
    for h in header:
        val=row[h]
        if h in distinct_value:
            values = distinct_value[h]
            if val in values:
                values.remove(val)
                distinct_value[h]=values
            if len(values)==0:
                del distinct_value[h]
df_train.to_csv("train.csv", sep=',')
df_test.to_csv("test.csv", sep=',')
print(df_train.shape)
print(df_test.shape)
## Delete Animal Name 

(174, 10)
(525, 10)


In [532]:
start_time = timeit.default_timer()
itree = decision_tree_infogain(df_train, 'class')
elapsed = timeit.default_timer() - start_time
size = getsizeof(itree)+getsizeof(decision_tree_infogain(df_train, 'class'))
print(itree)
print(elapsed, size)

['shape_uniformity']
	[1, 'clump_thickness']
		[5, 'size_uniformity']
			[1, 'marginal_adhesion']
				[1, 'epithelial_size']
					[2, 'bare_nucleoli']
						['1', 'bland_chromatin']
							[3, 'normal_nucleoli']
								[1, 'mitoses']
									[[1], 'class']
										[2]
									[[1], 'class']
										[2]
							[2, 'normal_nucleoli']
								[1, 'mitoses']
									[[1], 'class']
										[2]
						['2', 'bland_chromatin']
							[3, 'normal_nucleoli']
								[3, 'mitoses']
									[[1], 'class']
										[2]
						['?', 'bland_chromatin']
							[3, 'normal_nucleoli']
								[1, 'mitoses']
									[[1], 'class']
										[2]
			[3, 'marginal_adhesion']
				[2, 'epithelial_size']
					[2, 'bare_nucleoli']
						['1', 'bland_chromatin']
							[2, 'normal_nucleoli']
								[1, 'mitoses']
									[[1], 'class']
										[2]
			[2, 'marginal_adhesion']
				[1, 'epithelial_size']
					[2, 'bare_nucleoli']
						['1', 'bland_chromatin']
							[3, 'normal_nucleoli']
								[

In [ ]:
start_time = timeit.default_timer()
df["venomous"]
grtree = decision_tree_gainratio(df, 'venomous')
elapsed = timeit.default_timer() - start_time
size = getsizeof(grtree)+getsizeof(decision_tree_gainratio(df, "venomous"))
print(elapsed, size)
print(grtree)

In [ ]:
start_time = timeit.default_timer()
df["venomous"]
grtree = decision_tree_gini(df, 'venomous')
elapsed = timeit.default_timer() - start_time
size = getsizeof(grtree)+getsizeof(decision_tree_gainratio(df, "venomous"))
print(elapsed, size)
print(grtree)